### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.lines as mlines
import pandas as pd
import scipy.signal as ssl
from os import chdir
from matplotlib.ticker import MultipleLocator

import warnings
warnings.filterwarnings('ignore')
chdir("raman_data/")

fontsize = 12
plt.rcParams['font.size'] = fontsize

### Load methods

In [ ]:
def get_spectra_curve(x,intensities,frequencies,std):
    
    y = np.zeros(len(x))

    for i,intensity in enumerate(intensities):
        y = y + gaussian(x,intensity,frequencies[i],std)
    return x,y

def gaussian(x,amplitude,mean,std):
    return amplitude*np.exp(-(x-mean)**2/(2*std**2))

def get_max_peak(xdata,ydata,xmin,xmax):
    
    idx = np.where((xdata>xmin) & (xdata<xmax))[0]
    peaks = ssl.find_peaks(ydata[idx])[0]
    #print(peaks[0])
    biggest_peak_idx = peaks[np.argmax(ydata[idx][peaks])]
    biggest_peak_xdata = xdata[idx][biggest_peak_idx]
    biggest_peak_ydata = ydata[idx][biggest_peak_idx]
    return biggest_peak_xdata,biggest_peak_ydata

def get_max(xdata,ydata,xmin,xmax):
    
    idx = np.where((xdata>xmin) & (xdata<xmax))[0]
    max_idx = np.argmax(ydata[idx])
    max_xdata = xdata[idx][max_idx]
    max_ydata = ydata[idx][max_idx]
    return max_xdata,max_ydata

def rgb_to_cmyk(r, g, b):
    # Convert RGB values to range of 0-1
    r, g, b = r/255.0, g/255.0, b/255.0

    # Find the maximum value of RGB values
    max_value = max(r, g, b)

    # If max_value is 0, return 0, 0, 0, 1
    if max_value == 0:
        return 0, 0, 0, 1

    # Calculate the K value
    k = 1 - max_value

    # Calculate the C, M, and Y values
    c = (1 - r - k) / (1 - k)
    m = (1 - g - k) / (1 - k)
    y = (1 - b - k) / (1 - k)

    # Return the CMYK values
    return c, m, y, k

def cmyk_to_rgb(c, m, y, k):
    # Calculate the RGB values
    r = 255 * (1 - c) * (1 - k)
    g = 255 * (1 - m) * (1 - k)
    b = 255 * (1 - y) * (1 - k)

    # Round the RGB values and return them as integers
    return int(round(r)), int(round(g)), int(round(b))


def get_color(x,y):
    
    #datapoints = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]] #1-x,x,2-2y,2y
    
    valuepoints = np.array([[0.8, 0.8, 0.0, 0.2],  #Mo
                            [0.0, 0.0, 0.8, 0.2],  #W
                            [0.0, 0.8, 0.0, 0.2],  #S
                            [0.8, 0.0, 0.0, 0.2]]) #Se     in percentage
    
    cmyk_color = (x*valuepoints[0] + (1-x)*valuepoints[1] + y*valuepoints[2] + (1-y)*valuepoints[3])/2
    
    return tuple(np.array(cmyk_to_rgb(*cmyk_color))/255)

### MoS$\bf _{2-2y}$Se$\bf _{2y}$/WS$\bf _{2-2y}$Se$\bf _{2y}$ Raman plots

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(9.5,10),gridspec_kw={'width_ratios': [1.5, 1]})

In [ ]:
ys_exp = [0.00,0.28,0.33,0.47,0.52,0.67,1.0]
ys = [0.00,0.28,0.33,0.47,0.52,0.67,1.0,0.83,0.95]

x = 0

xspace = np.linspace(0,500,5000)
std = 3
nsamples = 15
offset = 2 # Distance between minimum of current composition from the max of previous composition
freq_scale = 1.035 #Frequency scaling to be multiplied for computed raman
last_max = 0 #Variable to track absolute y coordinate of previous composition

E_MoS2_theo = []
A1_MoS2_theo = []
A1_MoSe2_high_theo = []
A1_MoSe2_low_theo = []
E_MoSe2_theo = []



A1_MoS2_max = 410
E_MoS2_max = 385

npoly_guide = 2

xmin=130
xmax=490

# E_MoSe2_max = 268

for i,y in enumerate(sorted(ys)):
    
    color = get_color(x,y)
    
    if y in ys_exp:
    
        ### Plot experimental data 
        data = pd.read_csv(f'MoSSe/x{x:.3f}y{y:.3f}.csv',header=None)
        minmax_d = min(data[1])-last_max
        xdata_expt = data[0].values
        ydata_expt = (data[1]-minmax_d+offset).values
        last_max = max(ydata_expt)

        axs[0][0].fill_between(xdata_expt,ydata_expt,min(ydata_expt),alpha=0.4,color=color)
        axs[0][0].text(xmin+2,ydata_expt[0]+1,f'y = {y}')
        ###
    
    ### Plot calculated data
    nconfig = 1 if (y==0.0 or y==1.0) else nsamples
    avg_intens = np.zeros(len(xspace))
    for config in range(nconfig):
        npzfile = np.load(f'MoSSe_x{x:.3f}y{y:.3f}_config_{config}.npz')
        intens,freqs = npzfile['intensities'],npzfile['freqs']
        avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
    avg_intens /= nconfig
    
    #Scaling intensities such that max of calculated is equal to max of experimental
    if y in ys_exp:
        intens_scale = (max(data[1])-min(data[1]))/max(avg_intens)
    else:
        intens_scale=1
        ydata_expt=[0]
    xdata_theo = xspace*freq_scale
    ydata_theo = avg_intens*intens_scale+min(ydata_expt)
    
    if y in ys_exp:
        axs[0][0].plot(xdata_theo,ydata_theo,c=color)
    ###

    ##Track E' and A1 MoS2 in theory
    if y!= 1.0:
  
        
        A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,A1_MoS2_max-10,A1_MoS2_max)
        A1_MoS2_max = A1_x
        #E_x,E_y = get_max_peak(xdata_theo,ydata_theo,340,385)
        E_x,E_y = get_max_peak(xdata_theo,ydata_theo,E_MoS2_max-10,E_MoS2_max)
        E_MoS2_max = E_x
        E_MoS2_theo.append([E_x,E_y])
        A1_MoS2_theo.append([A1_x,A1_y])   
    
    ##Track two branches of splitted A1 MoSe2
    if y!=0.0:
        
        #High Frequency branch
        A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,240,280)
        #E_MoSe2_max = E_x
        A1_MoSe2_high_theo.append([A1_x,A1_y])
        
        #Low Frequency branch
        if y>0.33:
            A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,200,250)
            A1_MoSe2_low_theo.append([A1_x,A1_y])
            E_x,E_y = get_max_peak(xdata_theo,ydata_theo,280,300)
            E_MoSe2_theo.append([E_x,E_y])
            
            
    
    ##Label MoS2 E'and A1 peaks
    if y == 0:
        axs[0][0].text(E_x-14,E_y-8,r"MoS$_{2}$ like E'",ha='right')
        axs[0][0].arrow(E_x-12,E_y-4,8,3,width =0.3,color='Black')
        axs[0][0].text(A1_x+2,A1_y+6,r"MoS$_{2}$ like A$'_{1}$")
        axs[0][0].arrow(A1_x+10,A1_y+4,-7,-3,width=0.3,color='Black')
    
    ##Find MoSe2 A1 and E' peaks for text labelling
    if y == 1.0:
        A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,200,250)
        E_x,E_y = get_max_peak(xdata_theo,ydata_theo,250,300)
        axs[0][0].text(A1_x-12,A1_y+1,r"MoSe$_{2}$ like A$'_{1}$",ha='right')
        axs[0][0].arrow(A1_x-11,A1_y+2,7,-1,width =0.3,color='Black')
        axs[0][0].text(E_x+12,E_y+4,r"MoSe$_{2}$ like E'")
        axs[0][0].arrow(E_x+10,E_y+4,-7,-3,width=0.3,color='Black')
        
###Draw Guide Lines    
poly = np.polyfit([i[1] for i in E_MoS2_theo[:-2]],[i[0] for i in E_MoS2_theo[:-2]],npoly_guide)
E_MoS2_freqs = [np.polyval(poly,i[1]) for i in E_MoS2_theo[:-2]]
axs[0][0].plot(E_MoS2_freqs,[i[1] for i in E_MoS2_theo[:-2]],linestyle='--',c='b')

poly = np.polyfit([i[1] for i in A1_MoS2_theo[:-2]],[i[0] for i in A1_MoS2_theo[:-2]],npoly_guide)
A1_MoS2_freqs = [np.polyval(poly,i[1]) for i in A1_MoS2_theo[:-2]]
axs[0][0].plot(A1_MoS2_freqs,[i[1] for i in A1_MoS2_theo[:-2]],linestyle='--',c='r')

poly = np.polyfit([i[1] for i in A1_MoSe2_high_theo[:-3]],[i[0] for i in A1_MoSe2_high_theo[:-3]],npoly_guide)
A1_MoSe2_high_freqs = [np.polyval(poly,i[1]) for i in A1_MoSe2_high_theo[:-3]]
axs[0][0].plot(A1_MoSe2_high_freqs,[i[1] for i in A1_MoSe2_high_theo[:-3]],linestyle='--',c='g')

poly = np.polyfit([i[1] for i in A1_MoSe2_low_theo[:-3]],[i[0] for i in A1_MoSe2_low_theo[:-3]],npoly_guide)
A1_MoSe2_low_freqs = [np.polyval(poly,i[1]) for i in A1_MoSe2_low_theo[:-3]]
axs[0][0].plot(A1_MoSe2_low_freqs,[i[1] for i in A1_MoSe2_low_theo[:-3]],linestyle='--',c='m')

E_MoSe2_guide = [_ for i,_ in enumerate(E_MoSe2_theo) if i not in [3,4] ]
poly = np.polyfit([i[1] for i in E_MoSe2_guide],[i[0] for i in E_MoSe2_guide],npoly_guide)
E_MoSe2_freqs = [np.polyval(poly,i[1]) for i in E_MoSe2_guide]
axs[0][0].plot(E_MoSe2_freqs,[i[1] for i in E_MoSe2_guide],linestyle='--',c='y')

###    
    
axs[0][0].set_xlim(xmin,xmax)
axs[0][0].set_ylim(-5,123)
axs[0][0].set_yticks([])
axs[0][0].xaxis.set_minor_locator(MultipleLocator(10))

axs[0][0].set_xlabel(r'Raman shift (cm$^{-1}$)')
axs[0][0].set_ylabel('Intensity (a.u.)')
#axs[0][0].text(104,115,r'MoS$_{\bf 2-2x}$Se$_{\bf 2x}$',fontsize=fontsize+3,fontweight='extra bold')
axs[0][0].text(0.02,0.95,r'MoS$_{\bf 2-2y}$Se$_{\bf 2y}$',
         fontsize=fontsize+4,fontweight='extra bold',transform=axs[0][0].transAxes)


npoly = 2
ys = sorted(ys)

#E MoS2
E_MoS2_data = pd.read_csv(f'MoSSe/MoSSe_MoS2_E1.csv',header=None)
E_MoS2_data[0] = E_MoS2_data[0].round(decimals=2)
E_MoS2_data = E_MoS2_data.loc[E_MoS2_data[0].isin(ys_exp)][1].values
axs[0][1].plot(ys_exp[:-1],E_MoS2_data,marker='*',color='b',linestyle='--',
         markerfacecolor='none',label='Experiment')

poly = np.polyfit(ys[:-1],[i[0] for i in E_MoS2_theo],npoly)
E_MoS2_freqs = [np.polyval(poly,y) for y in ys[:-1]]
axs[0][1].plot(ys[:-1],E_MoS2_freqs,marker='*',color='b',label='Calculated')

axs[0][1].text(ys[0],E_MoS2_freqs[0]-19,r"MoS$_{2}$ like E'",color='b')

#A1 MoS2
A1_MoS2_data = pd.read_csv(f'MoSSe/MoSSe_MoS2_A1.csv',header=None)
A1_MoS2_data[0] = A1_MoS2_data[0].round(decimals=2)
A1_MoS2_data = A1_MoS2_data.loc[A1_MoS2_data[0].isin(ys_exp)][1].values
axs[0][1].plot(ys_exp[:-1],A1_MoS2_data,marker='o',color='r',linestyle='--',
         markerfacecolor='none',label='Experiment')

poly = np.polyfit(ys[:-1],[i[0] for i in A1_MoS2_theo],npoly)
A1_MoS2_freqs = [np.polyval(poly,y) for y in ys[:-1]]
axs[0][1].plot(ys[:-1],A1_MoS2_freqs,marker='o',color='r',label='Calculated')

axs[0][1].text(ys[0],A1_MoS2_freqs[0]-18,r"MoS$_{2}$ like A$'_{1}$",color='r')

#A1 MoSe2 high
A1_MoSe2_high_data = pd.read_csv(f'MoSSe/MoSSe_MoSe2_E1.csv',header=None)
A1_MoSe2_high_data[0] = A1_MoSe2_high_data[0].round(decimals=npoly)
A1_MoSe2_high_data = A1_MoSe2_high_data.loc[A1_MoSe2_high_data[0].isin(ys_exp)][1].values

axs[0][1].plot(ys_exp[1:-1],A1_MoSe2_high_data,marker='^',color='g',linestyle='--',
            markerfacecolor='none',label='Experiment')

poly = np.polyfit(ys[1:],[i[0] for i in A1_MoSe2_high_theo],npoly)
A1_MoSe2_high_freqs = [np.polyval(poly,y) for y in ys[1:]]
# axs[0][1].plot(sorted(ys[1:]),[y for x,y in sorted(zip(ys[1:],A1_MoSe2_high_freqs))],
#             marker='^',color='g',label='Calculated')
axs[0][1].plot(ys[1:],A1_MoSe2_high_freqs,
            marker='^',color='g',label='Calculated')

#A1 MoSe2 low
poly = np.polyfit(ys[3:],[i[0] for i in A1_MoSe2_low_theo],npoly)
A1_MoSe2_low_freqs = [np.polyval(poly,y) for y in ys[3:]]
axs[0][1].plot(ys[3:],A1_MoSe2_low_freqs,
            marker='^',color='m',label='Calculated')

axs[0][1].text(ys[1]-0.025,A1_MoSe2_low_theo[0][0]+20,r"MoSe$_{2}$ like A$'_{1}$",color='g')
axs[0][1].arrow(ys[2]+0.09,A1_MoSe2_low_freqs[0]+28,0.04,14,color='g',
          length_includes_head=True,head_width=0.012, head_length=3)
axs[0][1].arrow(ys[2]+0.09,A1_MoSe2_low_freqs[0]+18,0.04,-12,color='m',
          length_includes_head=True,head_width=0.012, head_length=3)

#E MoSe2
poly = np.polyfit(ys[3:],[i[0] for i in E_MoSe2_theo],npoly)
E_MoSe2_freqs = [np.polyval(poly,y) for y in ys[3:]]
axs[0][1].plot(ys[3:],E_MoSe2_freqs,
            marker='^',color='y',label='Calculated')
axs[0][1].text(ys[4]+0.025,E_MoSe2_theo[0][0]+3,r"MoSe$_{2}$ like E'",color='y')


experiment_line = mlines.Line2D([], [], color='black', marker='s',markerfacecolor='None',
                                linestyle='--',label='Experiment')
calculated_line = mlines.Line2D([], [], color='black', marker='s',label='Calculated')

axs[0][1].legend(handles=[experiment_line,calculated_line],loc='center left',
                 bbox_to_anchor=(0.0,0.6))

axs[0][1].set_ylabel(r'Raman shift (cm$^{-1}$)')
axs[0][1].set_xlabel('Se composition y')

axs[0][0].text(-0.08,0.95,'(a)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[0][0].transAxes)
axs[0][1].text(-0.23,0.95,'(b)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[0][1].transAxes)

In [ ]:
x = 1
ys_exp = [0.00,0.30,0.5,0.7,1.0]  
ys = [0.00,0.30,0.5,0.7,1.0,0.83,0.95]

xspace = np.linspace(0,500,5000)
std = 3
nsamples = 15
offset_distance = 1
freq_scale = 1.035
#last_max = 0 
npoly_guide = 2

A1_WS2_theo = []
A1_WSe2_theo = []
A1_WSe2_min = 250
A1_WSSe_theo = []
E_WS2_theo = []

last_max = 0

xmin=100
xmax=450

for i,y in enumerate(sorted(ys)):
    
    if y in ys_exp:
        color = get_color(x,y)
        #### Plot experimental data
        offset = 0 if y==0.0 else offset_distance
        data = pd.read_csv(f'WSSe/x{x:.3f}y{y:.3f}.csv',header=None)
        minmax_d = min(data[1])-last_max
        xdata_expt = data[0].values
        ydata_expt = (data[1]-minmax_d+offset).values
        last_max = max(ydata_expt)
        axs[1][0].fill_between(xdata_expt,ydata_expt,min(ydata_expt),alpha=0.4,color=color)
        axs[1][0].text(xmin+1,ydata_expt[0] +1 ,f'y = {y}')

    #### Plot calculated data
    nconfig = 1 if (y==0.0 or y==1.0) else nsamples
    avg_intens = np.zeros(len(xspace))
    for config in range(nconfig):
        npzfile = np.load(f'WSSe_x{x:.3f}y{y:.3f}_config_{config}.npz')
        intens,freqs = npzfile['intensities'],npzfile['freqs']
        avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
    avg_intens /= nconfig
    
    #Scaling intensities such that max of calculated is equal to max of experimental
    if y in ys_exp:
        intens_scale = (max(data[1])-min(data[1]))/max(avg_intens)
    else:
        intens_scale=1
        ydata_expt=[0]
    xdata_theo = xspace*freq_scale
    ydata_theo = avg_intens*intens_scale+min(ydata_expt)
    
    if y in ys_exp:
        axs[1][0].plot(xdata_theo,ydata_theo,c=color)
    
    ##Track A1 and E' WS2 in theory
    if y!= 1.0:
                
        A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,380,430)
        A1_WS2_theo.append([A1_x,A1_y])
        E_x,E_y = get_max_peak(xdata_theo,ydata_theo,340,370)
        E_WS2_theo.append([E_x,E_y])
          
    #Label WS2 E'and A1 peaks
    if y == 0:
        axs[1][0].text(E_x-14,E_y-8,r"WS$_{2}$ like E'",ha='right')
        axs[1][0].arrow(E_x-12,E_y-4,8,3,width =0.3,color='Black')
        axs[1][0].text(A1_x+1,-8,r"WS$_{2}$ like A$'_{1}$",ha='right')
        axs[1][0].arrow(A1_x-8,-2,5,3,width=0.3,color='Black')
        
    ##Track A1 WSe2 in theory
    if y!= 0.0:
        
        
        A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,A1_WSe2_min,270)
        A1_WSe2_min-=5
        A1_WSe2_theo.append([A1_x,A1_y])
        
    ##Label WSe2 A1 peak
    if y == 1.0:
        axs[1][0].text(A1_x-12,A1_y-12,r"WSe$_{2}$ like A$'_{1}$",ha='right')
        axs[1][0].arrow(A1_x-10,A1_y-6,6,4,width =0.3,color='Black')
        
    ##Track A1 WSSe in theory
    if y!= 0.0 and y!=1.0:
        A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,370,390)
        A1_WSSe_theo.append([A1_x,A1_y])

###Draw Guide Lines
poly = np.polyfit([i[1] for i in E_WS2_theo[:-2]],[i[0] for i in E_WS2_theo[:-2]],npoly_guide)
E_WS2_freqs = [np.polyval(poly,i[1]) for i in E_WS2_theo[:-2]]
axs[1][0].plot(E_WS2_freqs,[i[1] for i in E_WS2_theo[:-2]],linestyle='--',c='b')


poly = np.polyfit([i[1] for i in A1_WS2_theo[:-2]],[i[0] for i in A1_WS2_theo[:-2]],npoly_guide)
A1_WS2_freqs = [np.polyval(poly,i[1]) for i in A1_WS2_theo[:-2]]
axs[1][0].plot(A1_WS2_freqs,[i[1] for i in A1_WS2_theo[:-2]],linestyle='--',c='r')


poly = np.polyfit([i[1] for i in A1_WSe2_theo[:-2]],[i[0] for i in A1_WSe2_theo[:-2]],npoly_guide)
A1_WSe2_freqs = [np.polyval(poly,i[1]) for i in A1_WSe2_theo[:-2]]
axs[1][0].plot(A1_WSe2_freqs,[i[1] for i in A1_WSe2_theo[:-2]],linestyle='--',c='g')

#axs[1][0].plot([i[0] for i in A1_WSSe_theo],[i[1] for i in A1_WSSe_theo],linestyle='--',c='y')
poly = np.polyfit([i[1] for i in A1_WSSe_theo[:-2]],[i[0] for i in A1_WSSe_theo[:-2]],npoly_guide)
A1_WSSe_freqs = [np.polyval(poly,i[1]) for i in A1_WSSe_theo[:-2]]
axs[1][0].plot(A1_WSSe_freqs,[i[1] for i in A1_WSSe_theo[:-2]],linestyle='--',c='y')
###
    
axs[1][0].set_xlim(xmin,xmax)
axs[1][0].set_ylim(-10,125)
axs[1][0].set_yticks([])
axs[1][0].xaxis.set_minor_locator(MultipleLocator(10))
axs[1][0].set_xlabel(r'Raman shift (cm$^{-1}$)')
axs[1][0].set_ylabel('Intensity (a.u.)')
#axs[1][0].text(105,90,r'WS$_{\bf 2-2x}$Se$_{\bf 2x}$',fontsize=fontsize+3,fontweight='extra bold')
# axs[1][0].text(0.02,0.95,r'Mo$_{\bf 1-x}$W$_{\bf x}$Se$_{\bf 2}$',
#          fontsize=fontsize+3,fontweight='extra bold',transform=axs[1][0].transAxes)

axs[1][0].text(0.02,0.95,r'WS$_{\bf 2-2y}$Se$_{\bf 2y}$',
            fontsize=fontsize+4,fontweight='extra bold',transform=axs[1][0].transAxes)

npoly = 2 
ys = sorted(ys)

E_WS2_data = pd.read_csv(f'WSSe/WSSe_WS2_E1.csv',header=None)[1].values[::-1]
axs[1][1].plot(ys_exp[:-1],E_WS2_data,marker='*',color='b',linestyle='--',markerfacecolor='none',label='Experimental')


poly = np.polyfit(ys[:-1],[i[0] for i in E_WS2_theo],npoly)
E_WS2_freqs = [np.polyval(poly,y) for y in ys[:-1]]
axs[1][1].plot(ys[:-1],E_WS2_freqs,
            marker='*',color='b',label='Calculated')

axs[1][1].text(ys[0],E_WS2_theo[0][0]-15,r"WS$_{2}$ like E'",color='b')

A1_WS2_data = pd.read_csv(f'WSSe/WSSe_SW_A1.csv',header=None)[1].values[::-1]
axs[1][1].plot(ys_exp[:-1],A1_WS2_data,marker='o',color='r',linestyle='--',markerfacecolor='none',label='Experimental')

poly = np.polyfit(ys[:-1],[i[0] for i in A1_WS2_theo],npoly)
A1_WS2_freqs = [np.polyval(poly,y) for y in ys[:-1]]
axs[1][1].plot(ys[:-1],A1_WS2_freqs,
            marker='o',color='r',label='Calculated')

axs[1][1].text(ys[0],A1_WS2_theo[0][0]-18,r"WS$_{2}$ like A$'_{1}$",color='r')

A1_WSe2_data = pd.read_csv(f'WSSe/WSSe_SeW_A1.csv',header=None)[1].values[::-1]
axs[1][1].plot(ys_exp[1:],A1_WSe2_data,marker='^',color='g',linestyle='--',markerfacecolor='none',label='Experimental')

poly = np.polyfit(ys[1:],[i[0] for i in A1_WSe2_theo],npoly)
A1_WSe2_freqs = [np.polyval(poly,y) for y in ys[1:]]
axs[1][1].plot(ys[1:],A1_WSe2_freqs,
            marker='^',color='g',label='Calculated')

axs[1][1].text(ys[1]+0.09,A1_WSe2_theo[-1][0]+16,r"WSe$_{2}$ like A$'_{1}$",color='g')

A1_WSSe_data = pd.read_csv(f'WSSe/WSSe_SWSe_A1.csv',header=None)[1].values[::-1]
axs[1][1].plot(ys_exp[1:-1],A1_WSSe_data,marker='s',color='y',linestyle='--',markerfacecolor='none',label='Experimental')

poly = np.polyfit(ys[1:-1],[i[0] for i in A1_WSSe_theo],npoly)
A1_WSSe_freqs = [np.polyval(poly,y) for y in ys[1:-1]]
axs[1][1].plot(ys[1:-1],A1_WSSe_freqs,
            marker='s',color='y',label='Calculated')

axs[1][1].text(ys[1],A1_WSSe_theo[-1][0]-12,r"W-S-Se like A$'_{1}$",color='y')

experiment_line = mlines.Line2D([], [], color='black', marker='s',markerfacecolor='None',
                                linestyle='--',label='Experiment')
calculated_line = mlines.Line2D([], [], color='black', marker='s',label='Calculated')

axs[1][1].legend(handles=[experiment_line,calculated_line],loc='center right',
                 bbox_to_anchor=(0.6,0.3))

axs[1][1].set_ylabel(r'Raman shift (cm$^{-1}$)')
axs[1][1].set_xlabel('Se composition y')

axs[1][0].text(-0.08,0.95,'(c)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[1][0].transAxes)
axs[1][1].text(-0.26,0.95,'(d)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[1][1].transAxes)

In [ ]:
fig.tight_layout()
fig.subplots_adjust(wspace=0.22, hspace=0.2)

fig.savefig('MoSSe_WSSe_raman.png',facecolor='white', 
            transparent=False,dpi=300,bbox_inches='tight',pad_inches=0.02)
fig

### MoWS2 Raman

In [ ]:
#Compare MoWS2

xs = [0.00,0.20,0.42,0.53,0.61,0.77,1.0,0.83,0.95]
xs_exp = [0.00,0.20,0.42,0.53,0.61,0.77,1.0]

#xs = xs[::-1]
y = 0
plt.figure(figsize=(8,6))   
xspace = np.linspace(0,500,5000)
std = 1
nsamples = 15
offset = 2
freq_scale = 1.035
last_max = 0

d_theo = []

for i,x in enumerate(sorted(xs)):
    color = get_color(x,y)
    if x in xs_exp:
        
        ### Plot experimental data

        data = pd.read_csv(f'MoWS2/x{x:.3f}y{y:.3f}.csv',header=None)
        minmax_d = min(data[1])-last_max
        xdata_expt = data[0].values
        ydata_expt = (data[1]-minmax_d+offset).values
        last_max = max(ydata_expt)

        plt.fill_between(data[0],ydata_expt,min(ydata_expt),alpha=0.4,color=color)
        plt.text(min(xdata_expt),ydata_expt[0]+1,f'x = {x}')
    
    
    ### Plot calculated data
    nconfig = 1 if (x==0.0 or x==1.0) else nsamples
    avg_intens = np.zeros(len(xspace))
    
    for config in range(nconfig):

        npzfile = np.load(f'MoWS2_x{x:.3f}y{y:.3f}_config_{config}.npz')
        intens,freqs = npzfile['intensities'],npzfile['freqs']
        avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
    avg_intens /= nconfig
    
    if x in xs_exp:
        intens_scale = (max(data[1])-min(data[1]))/max(avg_intens)
    else:
        intens_scale=1
        ydata_expt=[0]
    
    xdata_theo = xspace*freq_scale
    ydata_theo = avg_intens*intens_scale+min(ydata_expt)
    
    if x in xs_exp:
        plt.plot(xdata_theo,ydata_theo,color=color)
    
    ##Track disorder related peak
    if x!=0 and x!=1:
        d_x,d_y = get_max_peak(xdata_theo,ydata_theo,355,365)
        d_theo.append([d_x,d_y])
    
plt.plot([i[0] for i in d_theo[:-2]],[i[1] for i in d_theo[:-2]],linestyle='--',c='purple')

plt.xlim(100,470)
plt.xlabel(r'Raman shift (cm$^{-1}$)')
plt.ylabel('Intensity (a.u.)')
plt.yticks([])
# plt.axes().xaxis.set_minor_locator(ml)
plt.text(105,90,r'Mo$_{\bf 1-x}$W$_{\bf x}$S$_{\bf 2}$',fontsize=fontsize+3,fontweight='extra bold')
plt.tight_layout()
plt.show()

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(9.5,10),gridspec_kw={'width_ratios': [1.5, 1]})

In [ ]:
xs = [0.00,0.20,0.42,0.53,0.61,0.77,1.0,0.83,0.95]
xs_exp = [0.00,0.20,0.42,0.53,0.61,0.77,1.0]

#xs = xs[::-1]
y = 0
    
xspace = np.linspace(0,500,5000)
std = 3
nsamples = 15
offset = 2
freq_scale = 1.035
last_max = 0
npoly = 2

A1_MoWS2_theo = []
E_MoS2_theo = []
E_WS2_theo = []

xmin = 250
xmax = 470

npoly_guide = 2

for i,x in enumerate(xs):
    
    color = get_color(x,y)
    ### Plot experimental data
    if x in xs_exp:
        data = pd.read_csv(f'MoWS2/x{x:.3f}y{y:.3f}.csv',header=None)
        minmax_d = min(data[1])-last_max
        xdata_expt = data[0].values
        ydata_expt = (data[1]-minmax_d+offset).values
        last_max = max(ydata_expt)

        axs[0][0].fill_between(data[0],ydata_expt,min(ydata_expt),alpha=0.4,color=color)
        axs[0][0].text(xmin+1,ydata_expt[0]+1,f'x = {x}')
    
    
    ### Plot calculated data
    nconfig = 1 if (x==0.0 or x==1.0) else nsamples
    avg_intens = np.zeros(len(xspace))
    
    for config in range(nconfig):

        npzfile = np.load(f'MoWS2_x{x:.3f}y{y:.3f}_config_{config}.npz')
        intens,freqs = npzfile['intensities'],npzfile['freqs']
        avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
    avg_intens /= nconfig
    
    if x in xs_exp:
        intens_scale = (max(data[1])-min(data[1]))/max(avg_intens)
    else:
        intens_scale=1
        ydata_expt=[0]

    
    xdata_theo = xspace*freq_scale
    ydata_theo = avg_intens*intens_scale+min(ydata_expt)
    
    if x in xs_exp:
        axs[0][0].plot(xdata_theo,ydata_theo,color=color)
    
    ##Track A1 in theory
    A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,390,450)
    A1_MoWS2_theo.append([A1_x,A1_y])
    
    ##Track E1 MoS2 in theory
    if x!=1.0:
        E_x,E_y = get_max_peak(xdata_theo,ydata_theo,360,385)
        E_MoS2_theo.append([E_x,E_y])
        
    #Label MoS2 E'and A1 peaks
    if x == 0:
        axs[0][0].text(E_x-14,E_y-8,r"MoS$_{2}$ like E'",ha='right')
        axs[0][0].arrow(E_x-12,E_y-4,8,3,width =0.3,color='Black')
        axs[0][0].text(A1_x+4,A1_y+4,r"A$'_{1}$")
        axs[0][0].arrow(A1_x+3,A1_y+4,-2,-2,width=0.3,color='Black')
    
    ##Track E1 WS2 in theory
    if x!=0.0:
        E_x,E_y = get_max_peak(xdata_theo,ydata_theo,330,355)
        E_WS2_theo.append([E_x,E_y])
        
    #Label WS2 A1 and E' peaks
    if x == 1.0:
        axs[0][0].text(E_x-14,E_y-8,r"WS$_{2}$ like E'",ha='right')
        axs[0][0].arrow(E_x-12,E_y-4,8,3,width =0.3,color='Black')
        axs[0][0].text(A1_x-9,A1_y+4,r"A$'_{1}$",ha='right')
        axs[0][0].arrow(A1_x-7,A1_y+3,4,-2,width=0.3,color='Black')

###Draw Guide Lines
poly = np.polyfit([i[1] for i in E_MoS2_theo[:-2]],[i[0] for i in E_MoS2_theo[:-2]],npoly_guide)
E_MoS2_freqs = [np.polyval(poly,i[1]) for i in E_MoS2_theo[:-2]]
axs[0][0].plot(E_MoS2_freqs,[i[1] for i in E_MoS2_theo[:-2]],linestyle='--',c='b')

poly = np.polyfit([i[1] for i in A1_MoWS2_theo[:-2]],[i[0] for i in A1_MoWS2_theo[:-2]],npoly_guide)
A1_MoWS2_freqs = [np.polyval(poly,i[1]) for i in A1_MoWS2_theo[:-2]]
axs[0][0].plot(A1_MoWS2_freqs,[i[1] for i in A1_MoWS2_theo[:-2]],linestyle='--',c='r')

poly = np.polyfit([i[1] for i in E_WS2_theo[:-2]],[i[0] for i in E_WS2_theo[:-2]],npoly_guide)
E_WS2_freqs = [np.polyval(poly,i[1]) for i in E_WS2_theo[:-2]]
axs[0][0].plot(E_WS2_freqs,[i[1] for i in E_WS2_theo[:-2]],linestyle='--',c='g')

poly = np.polyfit([i[1] for i in d_theo[:-2]],[i[0] for i in d_theo[:-2]],npoly_guide)
plot_values = [i[1] for i in d_theo[:-2]]
plot_values[0]-=1
plot_values[-1]+=5
d_freqs = [np.polyval(poly,i) for i in plot_values]
axs[0][0].plot(d_freqs,plot_values,linestyle='--',c='y')

axs[0][0].set_xlim(xmin,xmax)
axs[0][0].set_xlabel(r'Raman shift (cm$^{-1}$)')
axs[0][0].set_ylabel('Intensity (a.u.)')
axs[0][0].set_yticks([])

axs[0][0].xaxis.set_minor_locator(MultipleLocator(5))
#axs[0][0].text(xmin+5,90,r'Mo$_{\bf 1-x}$W$_{\bf x}$S$_{\bf 2}$',fontsize=fontsize+3,fontweight='extra bold')
axs[0][0].text(0.02,0.95,r'Mo$_{\bf 1-x}$W$_{\bf x}$S$_{\bf 2}$',
         fontsize=fontsize+4,fontweight='extra bold',transform=axs[0][0].transAxes)

#xs = sorted(xs,reverse=True)
xs_exp =xs_exp[::-1]
npoly = 2

E_MoS2_data = pd.read_csv(f'MoWS2/MoWS2_MoS2_E1.csv',header=None)
E_MoS2_data[0] = E_MoS2_data[0].round(decimals=2)
E_MoS2_data = E_MoS2_data.loc[E_MoS2_data[0].isin(xs)][1].values[::-1]
axs[0][1].plot(xs_exp[1:],E_MoS2_data,marker='*',color='b',linestyle='--',markerfacecolor='none',label='Experimental')

poly = np.polyfit(sorted(xs)[:-1],[i[0] for i in E_MoS2_theo],npoly)
E_MoS2_freqs = [np.polyval(poly,x) for x in sorted(xs)[:-1]]
axs[0][1].plot(sorted(xs)[:-1],E_MoS2_freqs,marker='*',color='b',label='Calculated')

axs[0][1].text(xs[0],E_MoS2_theo[0][0]-6,r"MoS$_{2}$ like E'",color='b')

A1_MoWS2_data = pd.read_csv(f'MoWS2/MoWS2_A1.csv',header=None)
A1_MoWS2_data[0] = A1_MoWS2_data[0].round(decimals=2)
A1_MoWS2_data = A1_MoWS2_data.loc[A1_MoWS2_data[0].isin(xs)][1].values[::-1]
axs[0][1].plot(xs_exp,A1_MoWS2_data,marker='o',color='r',linestyle='--',markerfacecolor='none',label='Experimental')

poly = np.polyfit(xs,[i[0] for i in A1_MoWS2_theo],npoly)
A1_MoWS2_freqs = [np.polyval(poly,x) for x in xs]
axs[0][1].plot(xs,A1_MoWS2_freqs,marker='o',color='r',label='Calculated')

axs[0][1].text(xs[0],A1_MoWS2_theo[0][0]+3,r"A$'_{1}$",color='r')

E_WS2_data = pd.read_csv(f'MoWS2/MoWS2_WS2_E1.csv',header=None)
E_WS2_data[0] = E_WS2_data[0].round(decimals=2)
E_WS2_data = E_WS2_data.loc[E_WS2_data[0].isin(xs)][1].values[::-1]
axs[0][1].plot(xs_exp[:-1],E_WS2_data,marker='^',color='g',linestyle='--',markerfacecolor='none',label='Experimental')


poly = np.polyfit(xs[1:],[i[0] for i in E_WS2_theo],npoly)
E_WS2_freqs = [np.polyval(poly,x) for x in xs[1:]]
axs[0][1].plot(xs[1:],E_WS2_freqs,marker='^',color='g',label='Calculated')

axs[0][1].text(xs[3],E_WS2_theo[2][0]-4,r"WS$_{2}$ like E'",color='g')


MoWS2_d_data = pd.read_csv(f'MoWS2/MoWS2_d.csv',header=None)
MoWS2_d_data[0] = MoWS2_d_data[0].round(decimals=2)
MoWS2_d_data = MoWS2_d_data.loc[MoWS2_d_data[0].isin(xs)][1].values[::-1]
axs[0][1].plot(xs_exp[1:-1],MoWS2_d_data,marker='D',color='y',linestyle='--',markerfacecolor='none',label='Experimental')

poly = np.polyfit(sorted(xs)[1:-1],[i[0] for i in d_theo],npoly)
d_freqs = [np.polyval(poly,x) for x in sorted(xs)[1:-1]]
axs[0][1].plot(sorted(xs)[1:-1],d_freqs,marker='D',color='y',label='Calculated')

axs[0][1].text(xs[1],d_theo[0][0]+2,"Disorder related mode",color='y')

experiment_line = mlines.Line2D([], [], color='black', marker='s',markerfacecolor='None',
                                linestyle='--',label='Experiment')
calculated_line = mlines.Line2D([], [], color='black', marker='s',label='Calculated')

axs[0][1].legend(handles=[experiment_line,calculated_line],loc='center right',bbox_to_anchor=(1,0.6))

axs[0][1].set_ylabel(r'Raman shift (cm$^{-1}$)')
axs[0][1].set_xlabel('W composition x')

axs[0][0].text(-0.08,0.95,'(a)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[0][0].transAxes)
axs[0][1].text(-0.26,0.95,'(b)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[0][1].transAxes)

In [ ]:
#Compare MoWSe2
xs = [0.00,0.21,0.39,0.53,0.73,0.81,1.0]
y = 1

xspace = np.linspace(0,500,5000)
std = 1.5
nsamples = 15
offset = 2
freq_scale = 1.035
last_max = 0

A1_MoWSe2_theo = []
npoly_guide = 2


for i,x in enumerate(xs):
    
    color = get_color(x,y)
    
    ### Plot experimental data
    data = pd.read_csv(f'MoWSe2/x{x:.3f}y{y:.3f}.csv',header=None)
    minmax_d = min(data[1])-last_max
    xdata_expt = data[0].values
    ydata_expt = (data[1]-minmax_d+offset).values
    last_max = max(ydata_expt)
    
    axs[1][0].fill_between(xdata_expt,ydata_expt,min(ydata_expt),alpha=0.4,color=color)
    axs[1][0].text(221,data[1][0]-minmax_d+offset+1,f'x = {xs[i]}')
    
    
    ### Plot calculated data
    nconfig = 1 if (x==0.0 or x==1.0) else nsamples
    avg_intens = np.zeros(len(xspace))
    for config in range(nconfig):
        npzfile = np.load(f'MoWSe2_x{x:.3f}y{y:.3f}_config_{config}.npz')
        intens,freqs = npzfile['intensities'],npzfile['freqs']
        avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
    avg_intens /= nconfig
    intens_scale = (max(data[1])-min(data[1]))/max(avg_intens)
    
    xdata_theo = xspace*freq_scale
    ydata_theo = avg_intens*intens_scale+min(ydata_expt)
    
    axs[1][0].plot(xdata_theo,ydata_theo,color=color)
  
    ##Track A1 in theory
    A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo,235,255)
    A1_MoWSe2_theo.append([A1_x,A1_y])
        
    #Label MoSe2 A1 peak
    if x == 0.0:
        #axs[1][0][0].text(A1_x-12,A1_y-1,r"MoSe$_{2}$ like A$'_{1}$")
        axs[1][0].text(A1_x-4,A1_y-1,r"A$'_{1}$",ha='right')
        axs[1][0].arrow(A1_x-3.5,A1_y,2,0,width =0.2,color='Black')

###Draw Guide Lines
poly = np.polyfit([i[1] for i in A1_MoWSe2_theo],[i[0] for i in A1_MoWSe2_theo],npoly_guide)
A1_MoWSe2_freqs = [np.polyval(poly,i[1]) for i in A1_MoWSe2_theo]
axs[1][0].plot(A1_MoWSe2_freqs,[i[1] for i in A1_MoWSe2_theo],linestyle='--',c='r')
###

axs[1][0].set_xlim(220,265)
axs[1][0].set_xlabel(r'Raman shift (cm$^{-1}$)')
axs[1][0].set_ylabel('Intensity (a.u.)')
axs[1][0].set_yticks([])

axs[1][0].text(0.02,0.95,r'Mo$_{\bf 1-x}$W$_{\bf x}$Se$_{\bf 2}$',
         fontsize=fontsize+4,fontweight='extra bold',transform=axs[1][0].transAxes)

npoly = 2
A1_MoWSe2_data = pd.read_csv(f'MoWSe2/MoWSe2_A1.csv',header=None)
A1_MoWSe2_data[0] = A1_MoWSe2_data[0].round(decimals=2)
A1_MoWSe2_data = A1_MoWSe2_data.loc[A1_MoWSe2_data[0].isin(xs)][1].values

poly = np.polyfit(xs,[i[0] for i in A1_MoWSe2_theo],npoly)
A1_MoWSe2_freqs = [np.polyval(poly,x) for x in xs]

axs[1][1].plot(xs,A1_MoWSe2_freqs,marker='o',color='r',label = 'Calculated')
axs[1][1].plot(xs,A1_MoWSe2_data,marker='o',color='r',
         linestyle='--',markerfacecolor='none',label='Experiment')
axs[1][1].text(xs[0],A1_MoWSe2_freqs[0]+1,r"A$'_{1}$",color='r')

experiment_line = mlines.Line2D([], [], color='black', marker='s',markerfacecolor='None',
                                linestyle='--',label='Experiment')
calculated_line = mlines.Line2D([], [], color='black', marker='s',label='Calculated')

axs[1][1].legend(handles=[experiment_line,calculated_line])

#plt.legend()
axs[1][1].set_ylabel(r'Raman shift (cm$^{-1}$)')
axs[1][1].set_xlabel('W composition x')

axs[1][0].text(-0.08,0.95,'(c)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[1][0].transAxes)
axs[1][1].text(-0.25,0.95,'(d)',fontsize=fontsize+4,fontweight='extra bold',transform=axs[1][1].transAxes)

In [ ]:
fig.tight_layout()
fig.savefig('MoWS2_MoWSe2_raman.png',facecolor='white', 
            transparent=False,dpi=300,bbox_inches='tight',pad_inches=0.02)
fig

### Reduced broadening for Mo$\bf _{1-x}$W${\bf  _x}$S$\bf _2$

In [ ]:
fig,axs = plt.subplots(figsize=(4,5))

# xs = [0.00,0.20,0.42,0.53,0.61,0.77,1.0,0.83,0.95]
# #xs_exp = [0.00,0.20,0.42,0.53,0.61,0.77,1.0]
# xs_exp = [0.20,0.42,0.53,0.61,0.77]
xs = [0.20,0.42,0.53,0.61,0.77]
# xs = xs[::-1]
y = 0
    
xspace = np.linspace(0,500,5000)
std = 1
nsamples = 15

freq_scale = 1.035
last_max = 0
npoly = 2

E_WS2_theo = []
d_theo = []

xmin = 330
xmax = 370
ymin=-5
ymax=21
npoly_guide = 2

for i,x in enumerate(xs):
    
    
    if x==0.42:
        offset= -8
    else:
        offset = -3.5
    
    color = get_color(x,y)
    ### Plot experimental data
#     if x in xs_exp:
    data = pd.read_csv(f'MoWS2/x{x:.3f}y{y:.3f}.csv',header=None)
    minmax_d = min(data[1])-last_max
    xdata_expt = data[0].values
    ydata_expt = (data[1]-minmax_d+offset).values
    last_max = max(ydata_expt)

    axs.fill_between(data[0],ydata_expt,min(ydata_expt),alpha=0.4,color=color)
    axs.text(xmin+1,ydata_expt[0]+0.5,f'x = {x}')
    
    
    ### Plot calculated data
    nconfig = 1 if (x==0.0 or x==1.0) else nsamples
    avg_intens = np.zeros(len(xspace))
    
    for config in range(nconfig):

        npzfile = np.load(f'MoWS2_x{x:.3f}y{y:.3f}_config_{config}.npz')
        intens,freqs = npzfile['intensities'],npzfile['freqs']
        avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
    avg_intens /= nconfig
    
    intens_scale = (max(data[1])-min(data[1]))/max(avg_intens)

    xdata_theo = xspace*freq_scale
    ydata_theo = avg_intens*intens_scale+min(ydata_expt)
    
    axs.plot(xdata_theo,ydata_theo,color=color)
        
    ##Track E1 WS2 in theory

    E_x,E_y = get_max_peak(xdata_theo,ydata_theo,330,355)
    E_WS2_theo.append([E_x,E_y])
    d_x,d_y = get_max_peak(xdata_theo,ydata_theo,355,365)
    d_theo.append([d_x,d_y])
    #Label WS2 A1 and E' peaks
    if x==0.77:
        axs.text(E_x-7,E_y-2,r"WS$_{2}$ like E'",ha='right')
        axs.arrow(E_x-6.5,E_y-1.3,4.5,1,width =0.2,color='Black')
        
        axs.text(d_x+3,d_y+1.9,"Disorder\n related\n mode",ha='center',va='bottom')
        axs.arrow(d_x+3,d_y+1.7,-1.2,-0.9,width =0.2,color='Black')
        
        
    


###Draw Guide Lines
# poly = np.polyfit([i[1] for i in E_WS2_theo[2:]],[i[0] for i in E_WS2_theo[2:]],npoly_guide)
# E_WS2_freqs = [np.polyval(poly,i[1]) for i in E_WS2_theo[2:]]
# axs.plot(E_WS2_freqs,[i[1] for i in E_WS2_theo[2:]],linestyle='--',c='g')

poly = np.polyfit([i[1] for i in E_WS2_theo],[i[0] for i in E_WS2_theo],npoly_guide)
E_WS2_freqs = [np.polyval(poly,i[1]) for i in E_WS2_theo]
axs.plot(E_WS2_freqs,[i[1] for i in E_WS2_theo],linestyle='--',c='g')

# poly = np.polyfit([i[1] for i in d_theo[2:]],[i[0] for i in d_theo[2:]],npoly_guide)
# d_freqs = [np.polyval(poly,i[1]) for i in d_theo[2:]]
# axs.plot(d_freqs,[i[1] for i in d_theo[2:]],linestyle='--',c='y')


poly = np.polyfit([i[1] for i in d_theo],[i[0] for i in d_theo],npoly_guide)
plot_values = [i[1] for i in d_theo]
# plot_values[0]-=1
# plot_values[-1]+=5
d_freqs = [np.polyval(poly,i) for i in plot_values]
axs.plot(d_freqs,plot_values,linestyle='--',c='y')
###




axs.set_xlim(xmin,xmax)
axs.set_ylim(ymin,ymax)
# axs.set_yticks([])
axs.set_xlabel(r'Raman shift (cm$^{-1}$)')
axs.set_ylabel('Intensity (a.u.)')


axs.xaxis.set_minor_locator(MultipleLocator(5))
#axs.text(xmin+5,90,r'Mo$_{\bf 1-x}$W$_{\bf x}$S$_{\bf 2}$',fontsize=fontsize+3,fontweight='extra bold')
axs.text(0.02,0.95,r'Mo$_{\bf 1-x}$W$_{\bf x}$S$_{\bf 2}$',
         fontsize=fontsize+4,fontweight='extra bold',transform=axs.transAxes)



plt.tight_layout()

plt.savefig('MoWS2_raman_std_1.png',facecolor='white', 
            transparent=False,dpi=300,bbox_inches='tight',pad_inches=0.02)

plt.show()

### Quaternary Raman plots

In [ ]:
xs = [0.25,0.5,0.75]
ys = [0.25,0.5,0.75]
xspace = np.linspace(0,500,5000)
std = 3
nsamples = 15
freq_scale = 1.035

fig,axs = plt.subplots(3,3,figsize=(11,10))

fontsize = 21.5
#fontsize = 4
plt.rcParams['font.size'] = fontsize
#plt.rcParams["figure.figsize"] = (4.75,3.75)

for i,x in enumerate(xs):
    for j,y in enumerate(ys):
        color = get_color(x,y)
        nconfig = nsamples
        avg_intens = np.zeros(len(xspace))
        for config in range(nconfig):
            npzfile = np.load(f'MoWSSe_x{x:.3f}y{y:.3f}_config_{config}.npz')
            intens,freqs = npzfile['intensities'],npzfile['freqs']
            avg_intens = avg_intens + get_spectra_curve(xspace,intens,freqs,std)[1]
        avg_intens /= nconfig
        
        xdata = xspace*freq_scale
        ydata = avg_intens
        axs[i][j].plot(xdata,ydata,color=color)
        axs[i][j].set_yticks([])
        axs[i][j].set_xlim(150,450)
        axs[i][j].set_ylim(-1,110)
        axs[i][j].xaxis.set_minor_locator(MultipleLocator(25))
        axs[i][j].xaxis.set_major_locator(MultipleLocator(100))
        
#         axs[i][j].text(0.05,30,'hy')
        axs[i][j].text(0.01,0.85,f'Mo$_{{{1-x}}}$W$_{{{x}}}$S$_{{{2-2*y}}}$Se$_{{{2*y}}}$',
                       fontsize=fontsize-2,transform=axs[i][j].transAxes)
        
        if i!=2:
            axs[i][j].tick_params(labelbottom=False)

#         if j==0:
#             axs[i][j].set_ylabel(f'x={x}')
        
#         if i==0:
#             axs[i][j].set_xlabel(f'y={y}')
#             axs[i][j].xaxis.set_label_position('top') 
            
fig.text(0.5, 0.055, r"Raman shift (cm$^{-1}$)", ha='center')
fig.text(0.09, 0.5, 'Intensity (a.u.)', va='center', rotation='vertical')

plt.subplots_adjust(wspace=0.05, hspace=0.1)
#plt.tight_layout()
plt.savefig('quaternary_raman.png',dpi=300,facecolor='white',transparent=False,
            bbox_inches='tight',pad_inches=0.02)
plt.show()

#### Comparison with MoWSSe

In [ ]:
x,y=0.5,0.5
color = get_color(x,y)
lstd=3
sstd=1
nsamples = 15
freq_scale = 1.035
xmin,xmax = 120,490
ymin,ymax = 0,5

xspace = np.linspace(0,500,5000)
data = pd.read_csv(f'MoWSSe/x{x:.3f}y{y:.3f}.csv',header=None)
minmax_d = min(data[1])
xdata_expt = data[0].values
ydata_expt = (data[1]-minmax_d).values

fontsize = 12.3
plt.rcParams['font.size'] = fontsize
plt.rcParams["figure.figsize"] = (4,3.5)
ml = MultipleLocator(10)
plt.axes().xaxis.set_minor_locator(ml)
plt.fill_between(data[0],ydata_expt,min(ydata_expt),alpha=0.4,color=color)

### Plot calculated data
nconfig = 1 if (x==0.0 or x==1.0) else nsamples
avg_intens_lstd = np.zeros(len(xspace))
avg_intens_sstd = np.zeros(len(xspace))
for config in range(nconfig):

    npzfile = np.load(f'MoWSSe_x{x:.3f}y{y:.3f}_config_{config}.npz')
    intens,freqs = npzfile['intensities'],npzfile['freqs']
    avg_intens_lstd = avg_intens_lstd + get_spectra_curve(xspace,intens,freqs,lstd)[1]
    avg_intens_sstd = avg_intens_sstd + get_spectra_curve(xspace,intens,freqs,sstd)[1]
avg_intens_lstd /= nconfig
avg_intens_sstd /= nconfig
intens_scale_lstd = (max(data[1])-min(data[1]))/max(avg_intens_lstd)
intens_scale_sstd = (max(data[1])-min(data[1]))/max(avg_intens_sstd)

xdata_theo = xspace*freq_scale
ydata_theo_lstd = avg_intens_lstd*intens_scale_lstd+min(ydata_expt)
ydata_theo_sstd = avg_intens_sstd*intens_scale_sstd+min(ydata_expt)

plt.plot(xdata_theo,ydata_theo_lstd,color=color)
plt.plot(xdata_theo,ydata_theo_sstd,color=color,lw=0.7,linestyle='--')

A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo_lstd,400,410)
plt.text(A1_x,A1_y+0.4,r"MoS$_2$/WS$_2$"+"\n"+"like A$'_{1}$",ha='center',va='bottom')
plt.arrow(A1_x,A1_y+0.4,0,-0.15,width=0.04,color='Black',
          head_width=3,
          head_length=0.1,
          head_starts_at_zero=True)
print(rf'MoS2/WS2 like A1 : {A1_x}')

E_x,E_y = get_max_peak(xdata_theo,ydata_theo_lstd,350,380)
plt.text(E_x-20,E_y+0.7,r"WS$_2$ like E$'$",ha='center')
plt.arrow(E_x,E_y+0.7,0,-0.45,width=0.04,color='Black',
          head_width=3,
          head_length=0.1,
          head_starts_at_zero=True)
print(rf"WS2 like E' : {E_x}")

E_x,E_y = get_max_peak(xdata_theo,ydata_theo_sstd,385,400)
plt.text(E_x+45,E_y+1.5,r"MoS$_2$"+"\n"+"like E$'$",ha='left',va='bottom')
plt.arrow(E_x+55,E_y+1.4,-50,-1.2,color='Black',
          head_width=0.1,
          head_length=2,
          head_starts_at_zero=True)
print(rf"MoS2 like E' : {E_x}")

A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo_lstd,260,270)
plt.text(A1_x-70,A1_y+1.7,r"MoSe$_2$/WSe$_2$"+"\n"+"like A$'_{1}$",ha='center',va='bottom')
plt.arrow(A1_x-70,A1_y+1.6,65,-1.5,color='Black',
          head_width=0.1,
          head_length=2)
print(rf'WSe2 like A1 : {A1_x}')

A1_x,A1_y = get_max_peak(xdata_theo,ydata_theo_lstd,240,260)
# plt.text(A1_x-62,A1_y+0.5,r"MoSe$_2$"+"\n"+r"like A$'_{1}$",ha='right',va='bottom')
# plt.arrow(A1_x-62,A1_y+0.4,55,-0.35,color='Black',
#           head_width=0.1,
#           head_length=2)
print(rf'MoSe2 like A1 : {A1_x}')

d_x,d_y = get_max_peak(xdata_theo,ydata_theo_sstd,324,334)
# plt.text(d_x-10,d_y+1.5,"Disorder\n related\n mode",ha='center')
# plt.arrow(d_x,d_y+1.4,0,-1.08,width=0.04,color='Black',
#           head_width=3,
#           head_length=0.1,
#           head_starts_at_zero=True)
print(rf'Disorder related mode : {d_x}')

plt.xlim(xmin,xmax)
plt.xlabel(r'Raman shift (cm$^{-1}$)')
plt.ylabel('Intensity (a.u.)')
plt.yticks([])
plt.ylim(ymin,ymax)

plt.text(0.01,0.9,rf'Mo$_{{\bf{1-x}}}$W$_{{\bf{x}}}$S$_{{\bf{2-2*y}}}$Se$_{{\bf{2*y}}}$',
               fontsize=fontsize,fontweight='extra bold',transform=plt.gca().transAxes)
# #plt.text(222,60,'$Mo_{1-x}W_{x}Se_{2}$',fontsize=20,fontweight='extra bold')
# #plt.text(222,60,r'Mo$_{\bf 1-x}$W$_{\bf x}$Se$_{\bf 2}$',fontsize=fontsize+3,fontweight='extra bold')
plt.tight_layout()


plt.savefig('MoWSSe_raman.png',dpi=300,facecolor='white',transparent=False,
            bbox_inches='tight',pad_inches=0.02)
plt.show()